#### Installs and Imports

In [ ]:
%%capture
!pip install selenium
!pip install thefuzz
!pip install python-Levenshtein
!pip install pyarabic

In [ ]:
import requests
from bs4 import BeautifulSoup
import csv
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
from thefuzz import fuzz
from thefuzz import process
from datetime import datetime
from pyarabic.araby import strip_tashkeel, normalize_hamza
import re
import numpy as np

#### Config

In [ ]:
pd.set_option('max_colwidth', 400)

In [ ]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}

#### Get Headlines

In [ ]:
def get_date(link):
    date_str = link.split("/")[3:6]
    date_str = "/".join(date_str)
    date_obj = datetime.strptime(date_str, "%Y/%m/%d")
    formatted_date = date_obj.strftime("%d/%m/%Y")
    return formatted_date

def get_headlines(page_url):
    headlines = []
    try:
        response = requests.get(page_url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')

        article_divs = soup.select('div.thumbnail-container.animate-lazy.size-715')

        for div in article_divs:
            link = div.find_parent('a', {'aria-label': True})
            if link:
                aria_label = link['aria-label']
                headline = aria_label.split('Read article:')[1].strip()
                headlines_date = get_date(link['href'])
                headlines.append((headlines_date, headline))

    except Exception as e:
        print(f"Error getting headlines from {page_url}: {str(e)}")
    return headlines

In [ ]:
base_url = "https://www.alborsaanews.com/category/%d8%a7%d9%84%d8%a8%d9%88%d8%b1%d8%b5%d8%a9-%d9%88%d8%a7%d9%84%d8%b4%d8%b1%d9%83%d8%a7%d8%aa/page/{}/"

all_headlines = []
for page_num in range(2, 100):
    page_url = base_url.format(page_num)
    print(f"Scraping page: {page_url}")

    headlines = get_headlines(page_url)
    all_headlines.extend(headlines)

    time.sleep(0.5)

with open('alborsa_headlines.csv', 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(['Date', 'Headline'])
    writer.writerows(all_headlines)
print(f"Scraped {len(all_headlines)} headlines")

Scraping page: https://www.alborsaanews.com/category/%d8%a7%d9%84%d8%a8%d9%88%d8%b1%d8%b5%d8%a9-%d9%88%d8%a7%d9%84%d8%b4%d8%b1%d9%83%d8%a7%d8%aa/page/2/
Scraping page: https://www.alborsaanews.com/category/%d8%a7%d9%84%d8%a8%d9%88%d8%b1%d8%b5%d8%a9-%d9%88%d8%a7%d9%84%d8%b4%d8%b1%d9%83%d8%a7%d8%aa/page/3/
Scraping page: https://www.alborsaanews.com/category/%d8%a7%d9%84%d8%a8%d9%88%d8%b1%d8%b5%d8%a9-%d9%88%d8%a7%d9%84%d8%b4%d8%b1%d9%83%d8%a7%d8%aa/page/4/
Scraping page: https://www.alborsaanews.com/category/%d8%a7%d9%84%d8%a8%d9%88%d8%b1%d8%b5%d8%a9-%d9%88%d8%a7%d9%84%d8%b4%d8%b1%d9%83%d8%a7%d8%aa/page/5/
Scraping page: https://www.alborsaanews.com/category/%d8%a7%d9%84%d8%a8%d9%88%d8%b1%d8%b5%d8%a9-%d9%88%d8%a7%d9%84%d8%b4%d8%b1%d9%83%d8%a7%d8%aa/page/6/
Scraping page: https://www.alborsaanews.com/category/%d8%a7%d9%84%d8%a8%d9%88%d8%b1%d8%b5%d8%a9-%d9%88%d8%a7%d9%84%d8%b4%d8%b1%d9%83%d8%a7%d8%aa/page/7/
Scraping page: https://www.alborsaanews.com/category/%d8%a7%d9%84%d8%a8%d9%88%d8%b

#### Mapping

In [ ]:
mapping_df = pd.read_csv('/content/drive/MyDrive/egx_listings.csv').merge(pd.read_excel('/content/drive/MyDrive/egx30.xlsx'))
headlines_df = pd.read_csv('alborsa_headlines.csv')

In [ ]:
headlines_df

,Date,Headline
0,02/03/2025,“أكت فاينانشال” تربح 355 مليون جنيه خلال 2024
1,02/03/2025,“جى بى كوربوريشن” تقترح توزيع 379.9 مليون جنيه على المساهمين
2,02/03/2025,“المتحدة للإسكان” تربح 130.3 مليون جنيه خلال 2024 بنمو 17.3%
3,02/03/2025,“فوري” تضاعف أرباحها السنوية إلى 1.74 مليار جنيه
4,02/03/2025,أرباح “جى بى كوربوريشن” تتخطى 3 مليارات جنيه خلال 2024
...,...,...
1955,13/09/2024,ارتفاع الأسهم الأوروبية عند الإغلاق محققة مكاسب أسبوعية
1956,13/09/2024,سهم “بوينج” ينخفض بعد فشل الاتفاق مع العمال المضربين
1957,13/09/2024,ارتفاع الأسهم الأمريكية مع تزايد احتمالات خفض الفائدة بوتيرة كبيرة
1958,13/09/2024,سهم “أدوبي” ينخفض 8% رغم تسجيل الشركة نتائج أعمال قوية


In [ ]:
mapping_df

,name_en,name_ar,id,category,code,weight
0,Abou Kir Fertilizers,ابوقير للاسمدة والصناعات الكيماوية,EGS38191C010,Basic Resources,ABUK.CA,3.88
1,Abu Dhabi Islamic Bank- Egypt,مصرف أبو ظبي الأسلامي- مصر,EGS60111C019,Banks,ADIB.CA,1.73
2,Alexandria Containers and goods,الاسكندرية لتداول الحاويات والبضائع,EGS42111C012,Shipping & Transportation Services,ALCN.CA,1.93
3,Alexandria Mineral Oils Company,الاسكندرية للزيوت المعدنية,EGS380P1C010,Energy & Support Services,AMOC.CA,1.39
4,Beltone Holding,بلتون القابضة,EGS691G1C015,Non-bank financial services,BTFH.CA,1.22
5,Commercial International Bank-Egypt (CIB),البنك التجاري الدولي -مصر (سى اى بى ),EGS60121C018,Banks,COMI.CA,26.75
6,Credit Agricole Egypt,بنك كريدي اجريكول مصر,EGS60041C018,Banks,CIEB.CA,1.86
7,E-finance For Digital and Financial Investments,اي فاينانس للاستثمارات المالية والرقمية,EGS743O1C013,"IT , Media & Communication Services",EFIH.CA,4.08
8,EFG Holding,مجموعة اي اف جي القابضة,EGS69101C011,Non-bank financial services,HRHO.CA,5.66
9,Eastern Company,الشرقية - ايسترن كومباني,EGS37091C013,"Food, Beverages and Tobacco",EAST.CA,8.55


In [ ]:
company_list = mapping_df['name_ar'].tolist()

def find_companies(row):
    text = row['Headline']

    matches = process.extractBests(
        text, company_list, scorer=fuzz.token_set_ratio, score_cutoff=75, limit=1
    )

    return [match[0] for match in matches][0] if matches else np.nan

In [ ]:
headlines_df['matched_companies'] = headlines_df.apply(find_companies, axis=1)

In [ ]:
headlines_df.dropna().value_counts('matched_companies')

,count
matched_companies,
المصرية للاتصالات,10
القابضة المصرية الكويتية,8
النساجون الشرقيون للسجاد,7
ابن سينا فارما,2
بلتون القابضة,2
جى بى كوربوريشن,2
القلعة للاستثمارات المالية,1
سيدى كرير للبتروكيماويات - سيدبك,1
مجموعة اي اف جي القابضة,1


In [ ]:
headlines_df[headlines_df.matched_companies == headlines_df['matched_companies'].value_counts().index[1]]

,Date,Headline,matched_companies
82,23/02/2025,أرباح “القابضة الكويتية” السنوية تتراجع 15% إلى 185.3 مليون دولار,القابضة المصرية الكويتية
283,03/02/2025,“أمان القابضة” توقع شراكة مع “جيديا” لدعم التجار في السوق المصرية,القابضة المصرية الكويتية
953,05/12/2024,تنفيذ صفقة على أسهم “القابضة الكويتية” بقيمة 367 مليون جنيه,القابضة المصرية الكويتية
1103,26/11/2024,تنفيذ صفقة على أسهم “القابضة الكويتية” بـ44.6 مليون جنيه,القابضة المصرية الكويتية
1182,18/11/2024,475 مليون دولار إيرادات “القابضة الكويتية” خلال أول 9 أشهر من 2024,القابضة المصرية الكويتية
1194,18/11/2024,“القابضة الكويتية” تضخ 247 مليون دولار استثمارات بقطاع الاستكشاف والتطوير,القابضة المصرية الكويتية
1207,17/11/2024,قطاع الأسمدة والبتروكيماويات يستحوذ على 55% من إيرادات «المصرية الكويتية»,القابضة المصرية الكويتية
1776,01/10/2024,هيكلة “القابضة الكويتية” تقود “بايونيرز” لصدارة سوق الصفقات بالربع الثالث,القابضة المصرية الكويتية


In [ ]:
headlines_df.matched_companies.value_counts()

,count
matched_companies,
المصرية للاتصالات,10
القابضة المصرية الكويتية,8
النساجون الشرقيون للسجاد,7
جى بى كوربوريشن,2
ابن سينا فارما,2
بلتون القابضة,2
مجموعة اي اف جي القابضة,1
القلعة للاستثمارات المالية,1
سيدى كرير للبتروكيماويات - سيدبك,1


In [ ]:
clean_headlines_df = headlines_df.dropna().merge(mapping_df, left_on='matched_companies', right_on='name_ar')

In [ ]:
clean_headlines_df.code = clean_headlines_df.code.str.replace('.CA', '.EGX')

In [ ]:
clean_headlines_df[clean_headlines_df.matched_companies == clean_headlines_df['matched_companies'].value_counts().index[1]]

,Date,Headline,matched_companies,name_en,name_ar,id,category,code,weight
1,19/02/2025,“المصرية للاتصالات” و”أورنج” توقعان اتفاقيات بقيمة 15 مليار جنيه,المصرية للاتصالات,Telecom Egypt,المصرية للاتصالات,EGS48031C016,"IT , Media & Communication Services",ETEL.EGX,4.04
6,24/12/2024,“المصرية للاتصالات” تعتمد موازنة 2025.. وتستهدف نمو الإيرادات بأكثر من 20%,المصرية للاتصالات,Telecom Egypt,المصرية للاتصالات,EGS48031C016,"IT , Media & Communication Services",ETEL.EGX,4.04
8,07/12/2024,هل تنجح زيادة الأسعار فى معادلة ضغوط التكلفة لدى “المصرية للاتصالات”؟,المصرية للاتصالات,Telecom Egypt,المصرية للاتصالات,EGS48031C016,"IT , Media & Communication Services",ETEL.EGX,4.04
14,18/11/2024,“المصرية للاتصالات” تحصل على شهادة التصميم Tier III لمركز البيانات الإقليمى,المصرية للاتصالات,Telecom Egypt,المصرية للاتصالات,EGS48031C016,"IT , Media & Communication Services",ETEL.EGX,4.04
17,17/11/2024,محمد نصر: “المصرية للاتصالات” استثمرت 102 مليار جنيه بالبنية التحتية خلال 5 سنوات,المصرية للاتصالات,Telecom Egypt,المصرية للاتصالات,EGS48031C016,"IT , Media & Communication Services",ETEL.EGX,4.04
18,14/11/2024,“المصرية للاتصالات” تربح 8.6 مليار جنيه خلال 9 أشهر,المصرية للاتصالات,Telecom Egypt,المصرية للاتصالات,EGS48031C016,"IT , Media & Communication Services",ETEL.EGX,4.04
24,13/10/2024,“المصرية للاتصالات” تعين وائل حنفي نائباً للرئيس التنفيذي للشؤون المالية,المصرية للاتصالات,Telecom Egypt,المصرية للاتصالات,EGS48031C016,"IT , Media & Communication Services",ETEL.EGX,4.04
25,07/10/2024,“تنظيم الاتصالات” يوقع ضوابط تقديم “المصرية للاتصالات” لخدمات الجيل الخامس,المصرية للاتصالات,Telecom Egypt,المصرية للاتصالات,EGS48031C016,"IT , Media & Communication Services",ETEL.EGX,4.04


In [ ]:
clean_headlines_df[clean_headlines_df.matched_companies == clean_headlines_df['matched_companies'].value_counts().index[1]].to_csv('ETEL.csv')

#### TODO
 1- Black Box

 2- Indexing by date

#### Stocks Analysis Agent

In [ ]:
!git clone https://github.com/Shubhamsaboo/awesome-llm-apps.git

Cloning into 'awesome-llm-apps'...
remote: Enumerating objects: 2434, done.
remote: Counting objects: 100% (86/86), done.
remote: Compressing objects: 100% (32/32), done.
remote: Total 2434 (delta 66), reused 54 (delta 54), pack-reused 2348 (from 2)
Receiving objects: 100% (2434/2434), 90.56 MiB | 39.23 MiB/s, done.
Resolving deltas: 100% (1166/1166), done.


In [ ]:
%cd awesome-llm-apps/ai_agent_tutorials/ai_investment_agent

/content/awesome-llm-apps/ai_agent_tutorials/ai_investment_agent


In [ ]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 69.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.2/487.2 kB 35.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 97.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.0/236.0 kB 20.1 MB/s eta 0:00:00


In [ ]:
!streamlit run investment_agent.py




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.58.159.219:8501

  Stopping...
  Stopping...
Exception ignored in: <module 'threading' from '/usr/lib/python3.11/threading.py'>
Traceback (most recent call last):
  File "/usr/lib/python3.11/threading.py", line 1560, in _shutdown
    atexit_call()
  File "/usr/lib/python3.11/concurrent/futures/thread.py", line 25, in _python_exit
    with _global_shutdown_lock:
  File "/usr/local/lib/python3.11/dist-packages/streamlit/web/bootstrap.py", line 44, in signal_handler
    server.stop()
  File "/usr/local/lib/python3.11/dist-packages/streamlit/web/server/server.py", line 470, in stop
    self._runtime.stop()
  File "/usr/local/lib/python3.11/dist-packages/streamlit/runtime/runtime.py", line 337, in stop
    async_objs.eventloop.call_soon_threadsafe(stop_on_eventloop)
  File "/usr/lib/python3.11/asyncio/base_events.py", line 807, in cal